### Computer Vision: Übungsblatt 3 <br> IMAGE STITCHING

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import cv2

#### a. Erweitern Sie Ihre Python-Funktion zur projektiven Entzerrung so, daß sie mehr als 4 Passpunkte verarbeiten kann.

In [ ]:
def projective_transform(source_image, source_points, target_shape, target_points):
    H = compute_homography_least_squares(source_points, target_points)
    output_image = warp_projective_indirect(source_image, H, target_shape)
    return output_image

def compute_homography_least_squares(source_points, target_points):
    """
    Computes the homography matrix H using the DLT algorithm and
    least-squares with pseudo-inverse for N >= 4 points.

    Args:
        source_points (np.ndarray): An Nx2 array of (x', y') coordinates.
        target_points (np.ndarray): An Nx2 array of (x, y) coordinates.

    Returns:
        np.ndarray: The 3x3 homography matrix H, or None if input is invalid.
    """
    n_points = source_points.shape[0]
    if n_points < 4 or source_points.shape != target_points.shape:
        print("Error: At least 4 point pairs are required and shapes must match.")
        return None

    # Build the M matrix and b vector
    M = []
    b = []
    for i in range(n_points):
        x_prime, y_prime = source_points[i]
        x, y = target_points[i]

        M.append([x_prime, y_prime, 1, 0, 0, 0, -x * x_prime, -x * y_prime])
        M.append([0, 0, 0, x_prime, y_prime, 1, -y * x_prime, -y * y_prime])
        b.append(x)
        b.append(y)

    M = np.array(M) # Shape (2N, 8)
    b = np.array(b) # Shape (2N,)

    # Solve Ma = b using the pseudo-inverse
    try:
        M_pinv = np.linalg.pinv(M)
        a = M_pinv @ b
    except np.linalg.LinAlgError:
        print("Error: Could not compute pseudo-inverse or solve the system.")
        return None

    # Reshape 'a' (which has 8 elements) into the 3x3 H matrix
    H = np.array([
        [a[0], a[1], a[2]],
        [a[3], a[4], a[5]],
        [a[6], a[7], 1.0]  # Add h33 = 1
    ])

    return H

# --- Example Usage ---
# Define more than 4 source and target points
# source_pts = np.array([[0, 0], [100, 0], [100, 100], [0, 100], [50, 50]])
# target_pts = np.array([[10, 10], [110, 5], [115, 110], [5, 105], [60, 60]])
#
# H_matrix = compute_homography_least_squares(source_pts, target_pts)
#
# if H_matrix is not None:
#     print("Computed Homography Matrix H:")
#     print(H_matrix)


def warp_projective_indirect(source_image, H_matrix, output_shape):
    """
    Warps a source image using a homography H via indirect mapping
    with nearest neighbor interpolation.

    Args:
        source_image (np.ndarray): The input image (H x W x C or H x W).
        H_matrix (np.ndarray): The 3x3 homography matrix.
        output_shape (tuple): A tuple (height, width) for the output image.

    Returns:
        np.ndarray: The warped output image.
    """
    source_height, source_width = source_image.shape[:2]
    output_height, output_width = output_shape

    # --- Step 1: Invert the Homography Matrix ---
    # We need H_inv to map from target back to source.
    try:
        H_inv = np.linalg.inv(H_matrix)
    except np.linalg.LinAlgError:
        print("Error: Homography matrix is not invertible.")
        return None

    # --- Step 2: Create the Output Image ---
    # Determine if it's color or grayscale
    if len(source_image.shape) == 3:
        output_image = np.zeros((output_height, output_width, source_image.shape[2]), dtype=source_image.dtype)
    else:
        output_image = np.zeros((output_height, output_width), dtype=source_image.dtype)

    # --- Step 3: Iterate Through Target Pixels (Indirect Mapping) ---
    for y_target in range(output_height):
        for x_target in range(output_width):

            # Create homogeneous coordinate for the target pixel
            target_coords_h = np.array([x_target, y_target, 1.0])

            # --- Step 4: Map Target to Source using H_inv ---
            source_coords_h = H_inv @ target_coords_h

            # --- Step 5: Convert back to Inhomogeneous Coordinates ---
            # Avoid division by zero or very small numbers
            w = source_coords_h[2]
            if abs(w) < 1e-9:
                continue

            x_source = source_coords_h[0] / w
            y_source = source_coords_h[1] / w

            # --- Step 6: Interpolate (Nearest Neighbor) ---
            x_s_int = int(round(x_source))
            y_s_int = int(round(y_source))

            # --- Step 7: Check Bounds and Assign Pixel Value ---
            if 0 <= x_s_int < source_width and 0 <= y_s_int < source_height:
                output_image[y_target, x_target] = source_image[y_s_int, x_s_int]

    return output_image

# --- Example Usage (requires an image and H) ---
# H = compute_homography_least_squares(source_pts, target_pts)
# try:
#     img = np.array(Image.open('your_image.jpg'))
# except FileNotFoundError:
#     print("Please replace 'your_image.jpg' with a real image file.")
#     img = None
#
# if H is not None and img is not None:
#     warped_img = warp_projective_indirect(img, H, (600, 800))
#     if warped_img is not None:
#         # Display or save the warped_img
#         # (e.g., using matplotlib or Pillow)
#         pass

In [ ]:
image_path = './../übungsblatt_2/schraegbild_tempelhof.jpg'

image = io.imread(image_path) # uint8 -> 0 - 255
print(type(image))
print(image.dtype)

image = image.astype(float)
# scale from 0 to 1
image /= 255.0


# Passpunkte
source_points_bi = np.array([
    [347, 328],  # b1 (x1, y1) - oben-links
    [537, 328],  # b2 (x2, y2) - oben-rechts
    [852, 538],  # b3 (x3, y3) - unten-rechts
    [266, 547]    # b4 (x4, y4) - unten-links
], dtype=float)

target_width = 400
target_height = 600

target_points_oi = np.array([
    [0, 0],
    [target_width - 1, 0],
    [target_width - 1, target_height - 1],
    [0, target_height - 1]
], dtype=float)


target_shape = (600, 400)

image = projective_transform(image, source_points_bi, target_shape, target_points_oi)
plt.imshow(image)
plt.show()

In [ ]:
image_path = './../übungsblatt_2/schraegbild_tempelhof.jpg'

image = io.imread(image_path) # uint8 -> 0 - 255
print(type(image))
print(image.dtype)

image = image.astype(float)
# scale from 0 to 1
image /= 255.0

# Passpunkte im Bild (source image coordinates)
source_points_bi = np.array([
    [347, 328],  # b1 - oben-links
    [537, 328],  # b2 - oben-rechts
    [820, 335],  # b3 - unten-rechts
    [887, 480],  # b4 - unten-links
    [440, 430],  # b5 - Mitte oben (zwischen b1 und b2)
    [560, 500],  # b6 - Mitte unten (zwischen b2 und b3)
], dtype=float)

# Zielkoordinaten im entzerrten Bild (target plane coordinates)
target_width = 800
target_height = 400

t_p = np.array([
    [0, 0],
    [int(target_width /2), 0],
    [target_width - 1, 0],
    [0, target_height - 1],
    [int(target_width / 2), int(target_height / 2)],
    [target_width - 1, target_height - 1],

], dtype=float)

target_points_oi = np.array([
    [0, 0],                                 # o1 - oben-links
    [target_width - 1, 0],                  # o2 - oben-rechts
    [0, 800],
    [10, 2],
    [target_width // 2, target_height // 3],       # o5 - Mitte oben
    [3 * target_width // 4, 2 * target_height // 3]  # o6 - Mitte unten
], dtype=float)


target_shape = (600, 400)

image = projective_transform(image, source_points_bi, target_shape, t_p)
plt.imshow(image)
plt.show()

#### b. Passpunkte und Weltkoordinaten Häuserfront

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_closed_polygon(points, color='blue', marker='o', label_prefix='P', show_labels=True, line_style='-', line_width=1):
    """
    Plottet eine Reihe von Punkten und verbindet sie sequentiell
    zu einem geschlossenen Polygon mit matplotlib.

    Args:
        points (np.ndarray): Ein Nx2 NumPy-Array mit den (x, y)-Koordinaten der Punkte.
        color (str): Die Farbe für die Punkte und Linien.
        marker (str): Der Marker-Stil für die Punkte (z.B. 'o', 'x', 's').
        label_prefix (str): Das Präfix für die Beschriftung der Punkte (z.B. 'b' oder 'o').
        show_labels (bool): Wenn True, werden die Punkte beschriftet.
        line_style (str): Der Linienstil (z.B. '-', '--', ':').
        line_width (float): Die Dicke der Linien.
    """
    # Überprüfen, ob genügend Punkte vorhanden sind
    if points is None or not isinstance(points, np.ndarray) or points.ndim != 2 or points.shape[1] != 2:
        print("Fehler: 'points' muss ein Nx2 NumPy-Array sein.")
        return

    num_points = points.shape[0]
    if num_points < 3:
        print("Warnung: Es werden mindestens 3 Punkte benötigt, um ein sinnvolles Polygon zu zeichnen.")
        # Wenn weniger als 3 Punkte, nur Punkte zeichnen
        plt.scatter(points[:, 0], points[:, 1], s=50, c=color, marker=marker, label=f'Punkte {label_prefix}')
        if show_labels:
            for i, (x, y) in enumerate(points):
                plt.text(x + 15, y - 15, f'${label_prefix}_{{{i+1}}}$', color=color, fontsize=12)
        return

    # 1. Punkte plotten (wie im Beispiel)
    plt.scatter(points[:, 0], points[:, 1], s=50, c=color, marker=marker, label=f'Passpunkte {label_prefix}')

    # 2. Text-Labels hinzufügen (wie im Beispiel)
    if show_labels:
        for i, (x, y) in enumerate(points):
            # Formatiert als LaTeX-String, z.B. $b_1$
            plt.text(x + 15, y - 15, f'${label_prefix}_{{{i+1}}}$', color=color, fontsize=12)

    # 3. Punkte für die Linien vorbereiten: Ersten Punkt am Ende anfügen
    x_lines = np.append(points[:, 0], points[0, 0])
    y_lines = np.append(points[:, 1], points[0, 1])

    # 4. Linien plotten (sequentiell + letzte zur ersten)
    plt.plot(x_lines, y_lines, color=color, linestyle=line_style, linewidth=line_width)




In [ ]:
image_all_path = './Fassade_ganz.jpg'
image_all = io.imread(image_all_path) # uint8 -> 0 - 255

image_all = image_all.astype(float)
# scale from 0 to 1
image_all /= 255.0

image_one_path = './Fassade_1.jpg'

image_one = io.imread(image_one_path) # uint8 -> 0 - 255

image_one = image_one.astype(float)
# scale from 0 to 1
image_one /= 255.0

target_points_1 = np.array([
[3820, 1780], [3380, 1820], [3330, 1140], [3080, 1400],[3610, 1790], [4260, 1520], [3460, 1500], [3110, 1500], [4450, 1810], [2740,1140],
[4140, 2260],
[4350, 1850]], dtype=float)


source_points_1 = np.array([
   [1630, 1220], [340, 1350], [1180, 220], [380, 680],[620, 1280], [1410, 610], [740, 790], [250, 840], [2130, 1180], [320, 360],
   [410, 2190],
   [1760, 1290]], dtype=float)

plt.figure(figsize=(12, 9))
plt.imshow(image_one)
plot_closed_polygon(source_points_1, color='red', label_prefix='b', show_labels=True)
plt.show()


image_two_path = './Fassade_2.jpg'

image_two = io.imread(image_two_path) # uint8 -> 0 - 255
image_two = image_two.astype(float)
image_two /= 255.0

target_points_2 = np.array([[3400, 1820], [3330, 1140], [3080, 1400],[2710, 1530], [2500, 1520], [2890, 2130]], dtype=float)
source_points_2 = np.array([[1800, 1840], [1450, 810], [1200, 1050], [580, 1120], [100, 1030], [910, 2380]], dtype=float)

plt.figure(figsize=(12, 9))
plt.imshow(image_two)
plot_closed_polygon(source_points_2, color='red', label_prefix='b', show_labels=True)
plt.show()

image_three_path = './Fassade_3.jpg'

image_three = io.imread(image_three_path) # uint8 -> 0 - 255

image_three = image_three.astype(float)
# scale from 0 to 1
image_three /= 255.0

target_points_3 = np.array([[3080, 1400],[2710, 1530], [1990, 1070], [1990, 2160], [1260, 1760], [2500, 1520], [2890, 2130],
                            [2140, 1320]], dtype=float)
source_points_3 = np.array([[3280, 630], [2440, 880], [1440, 210], [720, 2160],[310, 1470], [2020, 890], [2640, 2130],
                            [1320, 470]], dtype=float)

plt.figure(figsize=(12, 9))
plt.imshow(image_three)
plot_closed_polygon(source_points_3, color='red', label_prefix='b', show_labels=True)
plt.show()

image_four_path = './Fassade_4.jpg'
image_four = io.imread(image_four_path) # uint8 -> 0 - 255
image_four = image_four.astype(float)
# scale from 0 to 1
image_four /= 255.0
print('begin 1')
target_points_4 = np.array([[1990, 2160], [1260, 1760], [680, 1760], [1960, 1590], [1370, 1860], [2140, 1320]], dtype=float)
source_points_4 = np.array([[2430, 2630], [1650, 1970], [300, 1920], [2790, 1670], [1620, 2150], [3080, 1220]], dtype=float)

plt.figure(figsize=(12, 9))
plt.imshow(image_four)
plot_closed_polygon(source_points_4, color='red', label_prefix='b', show_labels=True)
plt.show()

# plt.figure(figsize=(18, 9))
# plt.imshow(image_all)
# plt.plot(target_points_1[:,0], target_points_1[:,1], color='red', linestyle='-', linewidth=1)
# plt.plot(target_points_2[:,0], target_points_2[:,1], color='blue', linestyle='-', linewidth=1)
# plt.plot(target_points_3[:,0], target_points_3[:, 1], color='green', linestyle='-', linewidth=1)
# plt.plot(target_points_4[:,0], target_points_4[:,1], color='yellow', linestyle='-', linewidth=1)
# plt.show()

#### c. Verschmelzen eines projektiv entzerrten Bildpaares

In [ ]:
def calculate_weights(height, width):
    """
    Calculates a weight map based on the distance from the center.
    Weights are 1.0 at the center and 0.0 at the edges.

    Args:
        height (int): The height of the image (N).
        width (int): The width of the image (M).

    Returns:
        np.ndarray: A (height, width) array with float weights.
    """
    # Create coordinate grids
    # M = width, N = height
    i_coords, j_coords = np.meshgrid(np.arange(width), np.arange(height))

    # Calculate weights using the formula
    # Note: Use 2.0 and M/2.0 to ensure float division
    w_i = 1.0 - (2.0 / width) * np.abs(i_coords - width / 2.0)
    w_j = 1.0 - (2.0 / height) * np.abs(j_coords - height / 2.0)

    # Combine weights and ensure they are >= 0 (due to float precision)
    weights = np.maximum(0.0, w_i * w_j)

    return weights

In [ ]:
import numpy as np

def blend_images(img1_warped, weights1_warped, img2_warped, weights2_warped, mode='average'):
    """
    Verschmilzt zwei transformierte Bilder anhand ihrer Gewichts-Maps. [cite: 5]
    Diese Funktion ist eine überarbeitete Version, um Fehler zu beheben
    und die Logik zu verdeutlichen.

    Args:
        img1_warped (np.ndarray): Das erste transformierte Bild (H, W, C oder H, W).
        weights1_warped (np.ndarray): Die transformierte Gewichts-Map für das erste Bild (H, W).
        img2_warped (np.ndarray): Das zweite transformierte Bild (H, W, C oder H, W).
        weights2_warped (np.ndarray): Die transformierte Gewichts-Map für das zweite Bild (H, W).
        mode (str): Der Modus: 'average' für gewichteten Mittelwert [cite: 10]
                    oder 'max' für die Auswahl des Pixels mit dem höheren Gewicht. [cite: 10]

    Returns:
        np.ndarray: Das verschmolzene Bild.
    """
    # --- 1. Eingabeüberprüfung ---
    if img1_warped.shape[:2] != weights1_warped.shape or \
       img2_warped.shape[:2] != weights2_warped.shape or \
       img1_warped.shape != img2_warped.shape:
        raise ValueError("Die Dimensionen von Bildern und Gewichten müssen übereinstimmen.")

    # --- 2. Vorbereitung ---
    # Konvertiere Bilder und Gewichte zu float für genaue Berechnungen
    img1_f = img1_warped.astype(np.float32)
    img2_f = img2_warped.astype(np.float32)
    w1_2d = weights1_warped.astype(np.float32)
    w2_2d = weights2_warped.astype(np.float32)

    # Ein kleiner Wert, um Division durch Null zu vermeiden
    epsilon = 1e-8

    # --- 3. Blending-Logik ---
    if mode == 'average':
        w1 = w1_2d[..., np.newaxis]
        w2 = w2_2d[..., np.newaxis]

        w_sum = w1 + w2 + epsilon
        blended_img = (img1_f * w1 + img2_f * w2) / w_sum

    elif mode == 'max':
        blended_img = np.zeros_like(img1_f)

        # Erzeuge 3D-Masken (H,W,1) für RGB-Bilder
        mask1 = (w1_2d >= w2_2d)[..., np.newaxis]
        mask2 = (w2_2d > w1_2d)[..., np.newaxis]


        blended_img = img1_f * mask1 + img2_f * mask2

    else:
        raise ValueError("Unbekannter Blending-Modus. Wähle 'average' oder 'max'.")

    # --- 4. Bereiche ohne Daten behandeln ---
    # Erstelle eine Maske für Bereiche, in denen *beide* Gewichte nahe Null sind
    no_data_mask = (w1_2d < epsilon) & (w2_2d < epsilon)
    # Setze diese Bereiche im finalen Bild auf Schwarz (0)
    blended_img[no_data_mask] = 0
    
    # --- 5. Rückgabe des verschmolzenen Bildes ---
    return np.clip(blended_img, 0, 255).astype(img1_warped.dtype)


In [ ]:
one_image_transformed = projective_transform(image_one, source_points_1, (2300, 4000), target_points_1)
plt.imshow(one_image_transformed)
plt.show()

two_image_transformed = projective_transform(image_two, source_points_2, (2300, 4000), target_points_2)
plt.imshow(two_image_transformed)
plt.show()

three_image_transformed = projective_transform(image_three, source_points_3, (2300, 4000), target_points_3)
plt.imshow(three_image_transformed)
plt.show()

four_image_transformed = projective_transform(image_four, source_points_4, (2300, 4000), target_points_4)
plt.imshow(four_image_transformed)
plt.show()

In [ ]:
h_one_orig, w_one_orig = one_image_transformed.shape[:2]
weights_one_orig = calculate_weights(h_one_orig, w_one_orig)
H_one = compute_homography_least_squares(source_points_1, target_points_1)
weights_transformed_one = cv2.warpPerspective(weights_one_orig, H_one, (4000, 2300))

h_two_orig, w_two_orig = two_image_transformed.shape[:2]
weights_two_orig = calculate_weights(h_two_orig, w_two_orig)
H_two = compute_homography_least_squares(source_points_2, target_points_2)
weights_transformed_two = cv2.warpPerspective(weights_two_orig, H_two, (4000, 2300))

h_three_orig, w_three_orig = three_image_transformed.shape[:2]
weights_three_orig = calculate_weights(h_three_orig, w_three_orig)
H_three = compute_homography_least_squares(source_points_3, target_points_3)
weights_transformed_three = cv2.warpPerspective(weights_three_orig, H_three, (4000, 2300))

h_four_orig, w_four_orig = four_image_transformed.shape[:2]
weights_four_orig = calculate_weights(h_four_orig, w_four_orig)
H_four = compute_homography_least_squares(source_points_4, target_points_4)
weights_transformed_four = cv2.warpPerspective(weights_four_orig, H_four, (4000, 2300))

In [ ]:
blended_image_max = blend_images(three_image_transformed, weights_transformed_three, two_image_transformed, weights_transformed_two, mode='max')
blended_image_ave = blend_images(three_image_transformed, weights_transformed_three, two_image_transformed, weights_transformed_two, mode='average')
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(blended_image_max)
plt.title("Pixel mit dem größeren Gewicht")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(blended_image_ave)
plt.title("gewichtetes Mittel aus allen Pixeln")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
blended_weights = weights_transformed_three + weights_transformed_two
blended_blended_image_max = blend_images(blended_image_max, blended_weights, four_image_transformed, weights_transformed_four, mode='max')
blended_blended_image_ave = blend_images(blended_image_ave, blended_weights, four_image_transformed, weights_transformed_four, mode='average')

blended_all_weights = blended_weights + weights_transformed_four
blended_all_image_max = blend_images(blended_blended_image_max, blended_all_weights, one_image_transformed, weights_transformed_one, mode='max')
blended_all_image_ave = blend_images(blended_blended_image_ave, blended_all_weights, one_image_transformed, weights_transformed_one, mode='average')


plt.figure(figsize=(12, 6))  

plt.subplot(1, 2, 1) 
plt.imshow(blended_all_image_max)
plt.title("Pixel mit dem größeren Gewicht")
plt.axis('off')

plt.subplot(1, 2, 2) 
plt.imshow(blended_all_image_ave)
plt.title("gewichtetes Mittel aus allen Pixeln")
plt.axis('off')

plt.tight_layout()
plt.show()

#### d. multi-band blending

In [ ]:
import numpy as np
import cv2 # Stellt sicher, dass OpenCV importiert ist

def multi_band_blend(img1_warped, weights1_warped, img2_warped, weights2_warped):
    """
    Verschmilzt zwei transformierte Bilder mittels Multi-Band Blending.
    Diese korrigierte Version behebt den IndexError bei der Maskierung.

    Args:
        img1_warped (np.ndarray): Das erste transformierte Bild.
        weights1_warped (np.ndarray): Die transformierte Gewichts-Map für das erste Bild (H, W).
        img2_warped (np.ndarray): Das zweite transformierte Bild.
        weights2_warped (np.ndarray): Die transformierte Gewichts-Map für das zweite Bild (H, W).

    Returns:
        np.ndarray: Das via Multi-Band Blending verschmolzene Bild.
    """
    # --- 1. Eingabeüberprüfung ---
    if img1_warped.shape[:2] != weights1_warped.shape or \
       img2_warped.shape[:2] != weights2_warped.shape or \
       img1_warped.shape != img2_warped.shape:
        raise ValueError("Die Dimensionen von Bildern und Gewichten müssen übereinstimmen.")

    # --- 2. Vorbereitung ---
    img1_f = img1_warped.astype(np.float32)
    img2_f = img2_warped.astype(np.float32)
    # Behalte 2D-Gewichte für Masken
    w1_2d = weights1_warped.astype(np.float32)
    w2_2d = weights2_warped.astype(np.float32)
    epsilon = 1e-8

    # --- 3. Zerlegung ---
    # Verwende Gauß-Filter für den Tiefpass [cite: 21]
    kernel_size_tuple = (201, 201)
    sigma_val = 20
    low1 = cv2.GaussianBlur(img1_f, kernel_size_tuple, sigma_val)
    low2 = cv2.GaussianBlur(img2_f, kernel_size_tuple, sigma_val)
    # Erhalte Hochpass durch Subtraktion [cite: 22]
    high1 = img1_f - low1
    high2 = img2_f - low2

    # --- 4. Tiefpass-Blending (Gewichteter Mittelwert) --- [cite: 13]
    # Erweitere Gewichte nur für die Mittelwert-Berechnung
    w1_bc = w1_2d[..., np.newaxis]
    w2_bc = w2_2d[..., np.newaxis]
    w_sum = w1_bc + w2_bc + epsilon
    blended_low = (low1 * w1_bc + low2 * w2_bc) / w_sum

    # --- 5. Hochpass-Blending (Maximales Gewicht) --- [cite: 13]
    blended_high = np.zeros_like(img1_f)
    # Erstelle 2D-Masken (H, W) aus den 2D-Gewichten
    mask_w1_ge_w2 = (w1_2d >= w2_2d)
    mask_w2_gt_w1 = (w2_2d > w1_2d)

    # Wende die 2D-Masken auf die 3D-Hochpassbilder an
    mask1 = mask_w1_ge_w2[..., np.newaxis]
    mask2 = mask_w2_gt_w1[..., np.newaxis]

    blended_high = high1 * mask1 + high2 * mask2


    # --- 6. Rekonstruktion ---
    blended_img = blended_low + blended_high

    # --- 7. Bereiche ohne Daten behandeln ---
    # Erstelle eine 2D-Maske für Bereiche, wo beide Gewichte Null sind
    no_data_mask = (w1_2d < epsilon) & (w2_2d < epsilon)
    # Wende die 2D-Maske an, um diese Bereiche auf Schwarz zu setzen
    blended_img[no_data_mask] = 0

    # --- 8. Finale Konvertierung ---
    return np.clip(blended_img, 0, 255).astype(img1_warped.dtype)

In [ ]:
multi_blended_image = multi_band_blend(three_image_transformed, weights_transformed_three, two_image_transformed, weights_transformed_two)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(multi_blended_image)
plt.title("multi band")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(blended_image_ave)
plt.title("gewichtetes Mittel aus allen Pixeln")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:

multi_blended_blended_image = multi_band_blend(multi_blended_image, blended_weights, four_image_transformed, weights_transformed_four)
multi_blended_all_image = multi_band_blend(multi_blended_blended_image, blended_all_weights, one_image_transformed, weights_transformed_one)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(blended_all_image_max)
plt.title("Gewichtung Max")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(blended_all_image_ave)
plt.title("gewichtetes Mittel aus allen Pixeln")
plt.axis('off')

plt.tight_layout()
plt.show()

plt.imshow(multi_blended_all_image)
plt.title("Multi Band Blending")
plt.axis('off')
plt.tight_layout()
plt.show()